In [30]:
!jt -t chesterish

In [31]:
import numpy as np
import pandas as pd
from pandas import read_csv



In [32]:
df_train_featureExtractionUnsorted = read_csv("SYSC4415W23_A3_dataset/train/extracted_features.csv")
df_train_labelsUnsorted = read_csv("SYSC4415W23_A3_dataset/train/labels.csv")


#remove NaN values replace with zero
df_train_featureExtractionUnsorted = df_train_featureExtractionUnsorted.fillna(0)
df_train_labelsUnsorted = df_train_labelsUnsorted.fillna(0)

# Replace infinite values with a valid value
df_train_featureExtractionUnsorted = df_train_featureExtractionUnsorted.replace([np.inf, -np.inf], np.nan)
df_train_featureExtractionUnsorted = df_train_featureExtractionUnsorted.fillna(df_train_featureExtractionUnsorted.max())

df_train_labelsUnsorted  = df_train_labelsUnsorted.replace([np.inf, -np.inf], np.nan)
df_train_labelsUnsorted  = df_train_labelsUnsorted.fillna(df_train_labelsUnsorted.max())



#sort numerically 
df_train_featureExtraction = df_train_featureExtractionUnsorted.sort_values('sample_id', ascending=True)
df_train_labels = df_train_labelsUnsorted.sort_values('sample_id', ascending=True)

df_train = pd.merge(df_train_featureExtraction, df_train_labels[['label']], left_index=True, right_index=True)




In [33]:
print(df_train.head())

      sample_id  az__variance_larger_than_standard_deviation  \
1018          1                                          1.0   
1533          2                                          1.0   
1346          3                                          1.0   
426           4                                          1.0   
60            6                                          1.0   

      az__has_duplicate_max  az__has_duplicate_min  az__has_duplicate  \
1018                    0.0                    0.0                1.0   
1533                    1.0                    1.0                1.0   
1346                    0.0                    0.0                1.0   
426                     0.0                    0.0                1.0   
60                      0.0                    0.0                1.0   

      az__sum_values  az__abs_energy  az__mean_abs_change  az__mean_change  \
1018      -11.405649     2958.427875             0.285573        -0.000523   
1533        4.430022

In [34]:
print(len(df_train))

1621


In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models



TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:


X = df_train.drop('label', axis=1).values
y = df_train['label'].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


# Reshape data to match input shape of CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define CNN architecture
model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
       
